## TinyImagenet training

### A. Target

Move your last code's transformations to Albumentations. Apply ToTensor, HorizontalFlip, Normalize (at min) + More (for additional points)
Please make sure that your test_transforms are simple and only using ToTensor and Normalize
Implement GradCam function as a module.
Your final code (notebook file) must use imported functions to implement transformations and GradCam functionality
Target Accuracy is 87%

### B. Results

Parameters: 11,173,962
Best Training Accuracy in 30 epochs: 96.51%
Best Test Accuracy in 30 epochs: 87.96 %
Total RF reached: 76*76 at the end of Conv block 4

### C. Analysis

I have implemented Albumentations transforms for normalization ( by finding norm and std values for entire dataset ), Horizontal flip, Vertical flip, Rotations. This acts as a regularizer and now the model is not overfitting to the extent it was earlier

I have also implemented the grad cam functionality and results are displayed for few of the images

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchsummary import summary
from torch.optim.lr_scheduler import OneCycleLR

import albumentations as A
from collections import Sequence
import numpy as np

from   torch.utils.data import Dataset
import torch.nn.functional as F

In [8]:
# !git clone https://github.com/SachinDangayach/EVA5
import sys

In [9]:
sys.path
sys.path.append(r'C:\Users\sdangayach162437\Desktop')

In [10]:
from tsai_eva5_work.session11.dataset import session11_dataset as dataset
from tsai_eva5_work.session11.models import session11 as models
from tsai_eva5_work.session11.models import session11_train_model as train
from tsai_eva5_work.session11.models import session11_test_model as test
from tsai_eva5_work.session9.utils import session9_utils as utils
from tsai_eva5_work.session11.utils import session11_lr_test as lr_test
from tsai_eva5_work.session10.utils import session10_lr_finder as lr_finder
from tsai_eva5_work.session9.utils import session9_view_gradcam as grad_cam

# Extract, Load and Transform Data

In [ ]:
import albumentations as A
import cv2
channel_means = (0.442,0.442,0.442)
channel_stdevs = (0.278, 0.278, 0.278)
train_transform = AlbumentationTransforms([       
                                                                
                                      A.HorizontalFlip(p = 0.7),
                                      A.PadIfNeeded(min_height=70, min_width=70, border_mode=4, value=None, mask_value=None, always_apply=False, p=1.0),
                                      A.RandomCrop(64, 64, always_apply=False, p=1.0),
                                      A.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
                                      A.Normalize(mean=channel_means, std=channel_stdevs),
                                      A.Cutout(num_holes=1, max_h_size=32,max_w_size = 32,p=0.7) 
                                       ])
test_transform = AlbumentationTransforms([A.Normalize(mean=channel_means, std=channel_stdevs)])
train_dataset , test_dataset,classes = TinyImageNetDataSet(train_split = 70,test_transforms = test_transform,train_transforms = train_transform)